In [1]:
%%HTML
<style>
.container { width: 100% }
</style>

# Embedded Actions in <span style="font-variant:small-caps;">Antlr</span> Grammars

Our grammar is stored in the file `Calculator.g4`.  The grammar describes a language for a symbolic calculator where we can store the results of our computations in variables.  This grammar makes use of <em style="color:blue">embedded actions</em>.

In [2]:
!cat Calculator.g4

grammar Calculator;

@header {
import math
}

start: statement+ ; 

statement
    : IDENTIFIER ':=' expr ';' {self.Values[$IDENTIFIER.text] = $expr.result}
    | expr ';'                 {print($expr.result)                         }
    ;

expr returns[result]
    : e=expr '+' p=product {$result = $e.result + $p.result}
    | e=expr '-' p=product {$result = $e.result - $p.result}
    | p=product            {$result = $p.result            }
    ;

product returns[result]
    : p=product '*' f=factor {$result = $p.result * $f.result}
    | p=product '/' f=factor {$result = $p.result / $f.result}
    | f=factor               {$result = $f.result            }
    ;

factor returns[result]
    : '(' expr ')'        {$result = $expr.result                 }
    | FLOAT               {$result = float($FLOAT.text)           }
    | IDENTIFIER          {$result = self.Values[$IDENTIFIER.text]}
    | 'sqrt' '(' expr ')' {$result = math.sqrt($expr.result)      }
    ;

IDENTIFIER: [a-zA-Z][a-zA-

First, we have to generate both the scanner and the parser.  

In [3]:
!antlr4 -Dlanguage=Python3 Calculator.g4

We can use the system command `ls` to see which files have been generated by <span style="font-variant:small-caps;">Antlr</span>.
If you are using a windows system you have to use the command `dir` instead.

In [4]:
!ls -l

total 160
-rw-r--r--@ 1 karlstroetmann  staff   1080 Jul 27 22:01 Calculator.g4
-rw-r--r--@ 1 karlstroetmann  staff   1146 Jul 27 19:20 Calculator.g4~
-rw-r--r--@ 1 karlstroetmann  staff   2643 Jul 27 22:03 Calculator.interp
-rw-r--r--@ 1 karlstroetmann  staff  18297 Jul 27 22:01 Calculator.ipynb
-rw-r--r--@ 1 karlstroetmann  staff    150 Jul 27 22:03 Calculator.tokens
-rw-r--r--@ 1 karlstroetmann  staff   2808 Jul 27 22:03 CalculatorLexer.interp
-rw-r--r--@ 1 karlstroetmann  staff   3407 Jul 27 22:03 CalculatorLexer.py
-rw-r--r--@ 1 karlstroetmann  staff    150 Jul 27 22:03 CalculatorLexer.tokens
-rw-r--r--@ 1 karlstroetmann  staff   1778 Jul 27 22:03 CalculatorListener.py
-rw-r--r--@ 1 karlstroetmann  staff  20374 Jul 27 22:03 CalculatorParser.py
drwxr-xr-x@ 6 karlstroetmann  staff    192 Jul 27 21:51 __pycache__
-rw-r--r--@ 1 karlstroetmann  staff     31 Jul 27 19:10 test.txt
-rw-r--r--@ 1 karlstroetmann  staff     14 Jul 27 18:10 test.txt~


The files `CalculatorLexer.py` and `CalculatorParser.py` contain the generated scanner and parser, respectively.  We have to import these files.  Furthermore, the runtime of 
<span style="font-variant:small-caps;">Antlr</span>
needs to be imported.

In [5]:
from CalculatorLexer  import CalculatorLexer
from CalculatorParser import CalculatorParser
import antlr4

First, let us parse the input that is stored in a file.  The function `parse_file` takes a filename `file`
as its argument.

In [6]:
def main():
    parser        = CalculatorParser(None)
    parser.Values = {}
    line          = input('> ')
    while line != '':
        input_stream  = antlr4.InputStream(line)
        lexer         = CalculatorLexer(input_stream)
        lexer.column  = 0
        token_stream  = antlr4.CommonTokenStream(lexer)
        parser.setInputStream(token_stream)
        parser.start()
        line = input('> ')
    return parser.Values

In [7]:
main()

> x := 3; y := 4;
> z := sqrt(x*x + y*y);
> z;
5.0
> 


{'x': 3.0, 'y': 4.0, 'z': 5.0}